# 2.3 Curry-Howard correspondence

In this notebook we deal with another striking connection between functional programming and, in principle, a completely unrelated field of knowledge. When algebraic data types were introduced, we established a strong connection with _arithmetics_. Now, it's the turn of _logic_. We will see that programming functionally is akin, in a very close sense, to doing logic. That, when we write a type signature we are writing propositions that have to be proved; and that when we implement that signature with some term expression, we are actually providing a proof of that proposition. Thus, this notebook resolves to show the following correspondences:

$$
\begin{array}{cc}
\mathrm{\bf Computation} & \mathrm{\bf Logic} \\
\hline 
\mathrm{Terms} & \mathrm{Proofs} \\
\mathrm{Types} & \mathrm{Propositions} 
\end{array}
$$

This correspondence between logic and computation is known as the Curry-Howard correspondence. It shows that programming functionally and following the laws of logic is the very same thing, ... and who on earth would want to program in a different way!

### References

[__The Curry-Howard correspondence__](https://en.wikipedia.org/wiki/Curry%E2%80%93Howard_correspondence) Wikpedia. 

[__Propositions as types__](https://homepages.inf.ed.ac.uk/wadler/papers/propositions-as-types/propositions-as-types.pdf). Philip Wadler. 

# A refresher: arithmetic and types

Recall the correspondence between algebraic data types and arithmetic operators: 

$$ 
\begin{array}{cc}
\mathrm{\bf Scala\ ADTs} & \mathrm{\bf Arithmetic} \\
\hline
\mathtt{Unit} & 1 \\
\mathtt{Nothing} & 0 \\
\mathtt{Either[X, Y]} & x + y \\
\mathtt{Tuple2[X, Y]} & x * y \\
\mathtt{X => Y} & {y^x}  
\end{array}
$$

According to this algebraic interpretation of types, the type 

In [41]:
type A = (Unit, Either[Unit, (Boolean => Either[Unit, Unit])])

defined type A

has exactly `1*(1+(1+1)^2)` values. But not only that: corresponding to the equality `1*(1+(1+1)^2)=5`, the following isomorphism holds: 

In [42]:
def from(a: A): Either[Unit, 
    Either[Unit, Either[Unit, Either[Unit, Unit]]]] = ???
def to(a: Either[Unit, 
    Either[Unit, Either[Unit, Either[Unit, Unit]]]]): A = ???

defined function from
defined function to

# Propositions as types

We put forward now the following correspondence: 

$$ 
\begin{array}{cc}
\mathrm{\bf Scala\ ADTs} & \mathrm{\bf Logic} \\
\hline
\mathtt{Nothing} & \bot \\
\mathtt{Either[P, Q]} & p \vee q \\
\mathtt{Tuple2[P, Q]} & p \wedge q \\
\mathtt{P => Q} & p \rightarrow q 
\end{array}
$$


so that the following type:

```scala
(P => Q, P) => Q
```

corresponds to the following formula of propositional logic: 

$\hspace{0.8cm}(p \rightarrow q) \wedge p \rightarrow q$

Let's get used to read Scala types as propositional formulas with these examples: 

$$
\begin{array}{cc}
\mathrm{\bf Scala\ types} & \mathrm{\bf Propositions} \\
\hline
\mathtt{(P, Either[Q, R])} & p \wedge (q\vee r) \\ 
\mathtt{P => Q => (Q, R)} & p \rightarrow q \rightarrow q \wedge r \\
\mathtt{(P => Nothing) => Nothing} & (p \rightarrow \bot) \rightarrow \bot \\ 
\end{array}
$$

What about logical negation and equivalence? They are commonly regarded as abbreviations: 

* $\neg p$ abbreviates $p \rightarrow \bot$
* $p \leftrightarrow q$ abbreviates $p \rightarrow q \wedge q \rightarrow p$

so, they will be represented in Scala through the following type aliases: 

In [43]:
type Not[P] = P => Nothing
type <=>[P, Q] = (P => Q, Q => P)

defined type Not
defined type <=>

Let's translate the following propositional formulas to Scala types: 


$$ 
\begin{array}{cc}
\mathrm{\bf Logic} & \mathrm{\bf Scala\ ADTs} \\
\hline
p \wedge q \rightarrow r \leftrightarrow p \rightarrow q \rightarrow r & \mathtt{(((P,Q)) => R) <=> (P => Q => R)} \\
\neg (p \wedge q) \rightarrow \neg p \vee \neg q & \mathtt{Not[(P, Q)] => Either[Not[P], Not[Q]]}\\
\neg\neg(p \vee \neg p) & \mathtt{Not[Not[Either[P, Not[P]]]]}\\
p \vee (q \wedge r) \leftrightarrow (p \vee q) \wedge (p \vee r) & 
\mathtt{Either[P, (Q, R)] <=> (Either[P, Q], Either[P, R])}
\end{array}
$$


Note that $p \wedge q \rightarrow r$ translates to `((P,Q)) => R`, and not to `(P, Q) => R`. The former is a `Function1` which receives a single argument of tuple type `(P, Q)`; the latter is a `Function2` which receives two arguments of types `P` and `Q`. 

# Proofs as terms

Right, but this is just a syntactic rewriting. In which sense, for instance, the Scala type

`(P, Either[Q, R]) => Either[(P, Q), (P, R)]`

justifies itself as a proper counterpart of the proposition
$p \wedge (q \vee r) \rightarrow p \wedge q \vee p \wedge r$? The answer is that they behave exactly in the same way: we can implement a program (expression or term) of that type if, and only if, the corresponding proposition is a theorem. Thus, the program is a witness that the proposition can be proved; in fact, we can automatically obtained a proof from that program, and viceversa. Let's start from a possible program:



In [44]:
def program[P, Q, R]: ((P, Either[Q, R])) => Either[(P, Q), (P, R)] = 
    t => t._2 match {
        case Left(q) => Left((t._1, q))
        case Right(r) => Right((t._1, r))
    }

defined function program

How is this program related to a proof of $p \wedge (q \vee r) \rightarrow p \wedge q \vee p \wedge r$? In order to talk about proofs we need a formal system, and our proof system of choice is Gentzen's natural deduction system: 

![natural deduction rules](../images/natdedrules.png)

Now, we can give a proof as follows:

$$
\begin{array}{llr}
1     & p \wedge (q \vee r) & premise \\
1.1   & p  & \wedge E(1) \\
1.2   & q \vee r & \wedge E(1) \\
&& \\
1.3   & q & premise \\
1.3.1 & p \wedge q & \wedge I(1.1,1.3) \\
1.3.2 & (p \wedge q) \vee (p \wedge r) & \vee I(1.3.1) \\
&& \\ 
1.4   & r & premise \\
1.4.1 & p \wedge r & \wedge I(1.1,1.4) \\
1.4.2 & (p \wedge q) \vee (p \wedge r) & \vee I(1.4.2) \\
&& \\
1.5   & (p \wedge q) \vee (p \wedge r) & \vee E(1.3-1.3.2,1.4-1.4.2,1.2) \\
2     & p \wedge (q \vee r) \rightarrow (p \wedge q) \vee (p \wedge r) & \rightarrow I(1-1.5)
\end{array}
$$

Apparently, this is at odds with the previous program, but algebraic data types also work in terms of introduction and ellimination rules, much in the same way as Gentzen's introduction and elimination rules. In order to emphasize this, let's reimplement our program in a more verbose way: 

In [108]:
def program[P, Q, R]: ((P, Either[Q, R])) => Either[(P, Q), (P, R)] = 
    {`1`: (P, Either[Q,R])  =>                        // 1     p ∧ (q ∨ r) 
        val `1.1`: P = `1`._1                         // 1.1   p 
        val `1.2`: Either[Q, R] = `1`._2              // 1.2   q ∨ r 
        (`1.2` match {                                
            case Left(_1_3: Q) =>                     // 1.3   q
                val `1.3.1`: (P, Q) = (`1.1`,_1_3)    // 1.3.1 p ∧ q 
                Left(`1.3.1`)                         // 1.3.2 (p ∧ q) ∨ (p ∧ r)
            
            case Right(_1_4: R) =>                    // 1.4   r
                val `1.4.1`: (P, R) = (`1.1`, _1_4)   // 1.4.1 p ∧ r 
                Right(`1.4.1`)                        // 1.4.2 (p ∧ q) ∨ (p ∧ r)
            
        }): Either[(P,Q), (P,R)]                      // 1.5   (p ∧ q) ∨ (p ∧ r)
    }: (((P, Either[Q, R])) => Either[(P,Q), (P,R)])  // 2     p ∧ (q ∨ r) ⟶ (p ∧ q) ∨ (p ∧ r)

defined function program

As we can see, conjunction behave very much like products, in the sense that the introduction rule $\wedge.I$ corresponds to the product constructor `(_, _)`, and the elimination rule $\wedge.E$ to the selectors `_._1` and `_._2`. Similarly, the introduction rule for disjunction corresponds to the constructors of `Either`, i.e. `Left(_)` and `Right(_)`, and the elimination rule to pattern matching. Last, the introduction of an implication corresponds to the contruction of a lambda expression, and its ellimination (not shown in the example), to function application. We can obtain an even more closer correspondence between the program and its corresponding proof with the following type aliases and definitions:

In [109]:
object Ax{
    def apply[P](a: P): P = a
}

type ⟶[A, B] = A => B

object ⟶{
    def I[A, B](f: A => B): A ⟶ B =
        f
    
    def E[A, B](f: A ⟶ B)(a: A): B = 
        f(a)
}

type ⊥ = Nothing 

object ⊥{
    def E[A](a: ⊥): A = a
} 

type ∨[A, B] = Either[A, B]

object ∨{
    def IL[A, B](a: A): A ∨ B = Left(a)

    def IR[A, B](b: B): A ∨ B = Right(b)
    
    def E[A, B, C](f: A => C, g: B => C)(or: A ∨ B): C = 
        or match {
            case Left(a: A) => f(a)
            case Right(b: B) => g(b)
        }
}

type ∧[A, B] = (A, B)

object ∧{
    def I[A, B](a: A, b: B): A ∧ B = (a, b)
    
    def EL[A, B](and: A ∧ B): A = and._1
    
    def ER[A, B](and: A ∧ B): B = and._2
}

defined object Ax
defined type ⟶
defined object ⟶
defined type ⊥
defined object ⊥
defined type ∨
defined object ∨
defined type ∧
defined object ∧

Together, the fully implement the introduction and ellimination rules of the natural deduction system introduced above. We can also add these aliases for negation and equivalence. 

In [110]:
type ¬[A] = A ⟶ ⊥
type ⟷[A, B] = (A ⟶ B) ∧ (B ⟶ A)

defined type ¬
defined type ⟷

The following aliases will also come in handy if we can't easily write unicode characters in our keyboard:

In [111]:
type Absurd = ⊥
type Or[A, B] = ∨[A, B] 
type And[A, B] = ∧[A, B]
type Not[A] = ¬[A]
type <=>[A, B] = (A => B, B => A)
val Implies = ⟶
val Or = ∨ 
val And = ∧

defined type Absurd
defined type Or
defined type And
defined type Not
defined type <=>
Implies: ⟶.type = ammonite.$sess.cmd108$Helper$$u27F6$@798cef90
Or: ∨.type = ammonite.$sess.cmd108$Helper$$u2228$@5cdfff20
And: ∧.type = ammonite.$sess.cmd108$Helper$$u2227$@6cc80fb5

Now, we can re-implement our program so that its correspondence with the natural deduction proof is more than apparent:

In [112]:
def proof[P, Q, R]: (P ∧ (Q ∨ R)) ⟶ ((P ∧ Q) ∨ (P ∧ R)) = 
    ⟶.I{ `1`: P ∧ (Q ∨ R) =>                          // 1     p ∧ (q ∨ r) 
        val `1.1`: P = ∧.EL(`1`)                      // 1.1   p 
        val `1.2`: Q ∨ R = ∧.ER(`1`)                  // 1.2   q ∨ r 
        ∨.E(
            { `1.3`: Q =>                             // 1.3   q
              val `1.3.1`: P ∧ Q = ∧.I(`1.1`, `1.3`)  // 1.3.1 p ∧ q 
              ∨.IL(`1.3.1`)                           // 1.3.2 (p ∧ q) ∨ (p ∧ r)
            }, 
            { `1.4`: R =>                             // 1.4   r
              val `1.4.1`: P ∧ R = ∧.I(`1.1`, `1.4`)  // 1.4.1 p ∧ r 
              ∨.IR(`1.4.1`)                           // 1.4.2 (p ∧ q) ∨ (p ∧ r)
            })(
            `1.2`): (P ∧ Q) ∨ (P ∧ R)                 // 1.5   (p ∧ q) ∨ (p ∧ r)
    } : (P ∧ (Q ∨ R)) ⟶ ((P ∧ Q) ∨ (P ∧ R))           // 2     p ∧ (q ∨ r) ⟶ (p ∧ q) ∨ (p ∧ r)

defined function proof

# More examples

If you can't still believe what we have shown, let's see some more examples which show how to prove a theorem with a functional program.

# $\vdash (p \rightarrow q) \wedge p \rightarrow q$

In [50]:
def program[P, Q]: (P => Q, P) => Q = 
    //(x, f) => f(x)
    _(_)

defined function program

That this program represents exactly this proof:

$$
\begin{array}{llr}
1 & (p \rightarrow q) \wedge p & premise \\
1.1 & p \rightarrow q & \wedge E(1) \\
1.2 & p  & \wedge E(1) \\
1.3 & q  & \rightarrow E(1.1,1.2) \\
2 & (p \rightarrow q) \wedge p \rightarrow q & \rightarrow I(1-1.3)
\end{array}
$$

can be shown more properly in the following way:

In [128]:
def proof[P, Q]: (P ⟶ Q) ∧ P ⟶ Q = 
    ⟶.I{ `1`: (P ⟶ Q) ∧ P =>            // 1   (p ⟶ q) ∧ p
        val `1.1`: P ⟶ Q = ∧.EL(`1`)    // 1.1 p ⟶ q
        val `1.2`: P = ∧.ER(`1`)        // 1.2 p
        ⟶.E(`1.1`)(`1.2`): Q            // 1.3 q
    }: (P ⟶ Q) ∧ P ⟶ Q                  // 2   (p ⟶ q) ∧ p ⟶ q

defined function proof

# $\vdash p \vee q \rightarrow \neg p \rightarrow q$ 

In [129]:
def proof[P, Q]: Either[P, Q] => (P => Nothing) => Q = 
    (e: Either[P, Q]) => 
        (f: P => Nothing) => 
            e match {
                case Left(a: P) => f(a): Q
                case Right(b: Q) => b : Q
            }

defined function proof

That this program represents exactly this proof:

$$
\begin{array}{llr}
1 & p \vee q & \mathrm{premise} \\ 
1.1 & \neg p & \mathrm{premise} \\
&& \\
1.1.1 & p & \mathrm{premise} \\
1.1.1.1 & \bot & \rightarrow E(1.1,1.1.1) \\
1.1.1.2 & q & \bot E(1.1.1.1)\\
&& \\
1.1.2 & q & \mathrm{premise} \\
1.1.2.1 & q & \mathrm{Ax}(1.1.2) \\
&& \\
1.1.3 & q & \vee E(1.1.1-1.1.1.2,1.1.2-1.1.2.1,1)\\ 
1.2 & \neg p \rightarrow q & \rightarrow I(1.1-1.1.3)\\
2 & p \vee q \rightarrow \neg p \rightarrow q & \rightarrow I(1-1.2) 
\end{array}
$$

can be shown more properly in the following way:

In [130]:
def proof[P, Q]: P ∨ Q ⟶ (¬[P] ⟶ Q) = 
    ⟶.I{ `1`: P ∨ Q =>                                            // 1       p ∨ q
        ⟶.I{ `1.1`: ¬[P] =>                                       // 1.1     ¬p
            ∨.E(
                { `1.1.1`: P =>                                   // 1.1.1   p
                    val `1.1.1.1`: ⊥ = ⟶.E[P, ⊥](`1.1`)(`1.1.1`)  // 1.1.1.1 ⊥ 
                    ⊥.E(`1.1.1.1`): Q                             // 1.1.1.2 q
                },
                { `1.1.2`: Q =>                                   // 1.1.2   q   
                    Ax(`1.1.2`)                                   // 1.1.2.1 q
                })(
                `1`): Q                                           // 1.1.3   q
        }: ¬[P] ⟶ Q                                               // 1.2     ¬p ⟶ q
    }: P ∨ Q ⟶ (¬[P] ⟶ Q)                                         // 2       p ∨ q ⟶ ¬p ⟶ q

defined function proof

# $\vdash (p \rightarrow \neg q) \rightarrow (p \rightarrow q) \rightarrow \neg p$

In [131]:
def proof3[P, Q]: (P => Not[Q]) => (P => Q) => Not[P] = 
    (a1: P => Not[Q]) => 
        (a2: P => Q) => 
            (a: P) => a1(a)(a2(a)) : Nothing

defined function proof3

That this program represents exactly this proof:

$$
\begin{array}{llr}
1 & p \rightarrow \neg q & premise \\
1.1 & p \rightarrow q & premise \\
1.1.1 & p & premise \\ 
1.1.1.1 & \neg q & \rightarrow E(1,1.1.1) \\
1.1.1.2 & q & \rightarrow E(1.1, 1.1.1) \\
1.1.1.3 & \bot & \rightarrow E(1.1.1.1,1.1.1.2) \\
1.1.2 & \neg p & \rightarrow I(1.1.1-1.1.1.3) \\
1.2 & (p \rightarrow q) \rightarrow \neg p & \rightarrow I(1.1-1.1.2)\\
2 & (p \rightarrow \neg q) \rightarrow (p \rightarrow q) \rightarrow \neg p & \rightarrow I(1-1.2)
\end{array}
$$

can be shown more properly in the following way:

In [132]:
def proof3[P, Q]: (P ⟶ ¬[Q]) ⟶ ((P ⟶ Q) ⟶ (¬[P])) = 
    ⟶.I{ `1`: (P ⟶ ¬[Q]) =>                             // 1        p ⟶ ¬q
        ⟶.I{ `1.1`: (P ⟶ Q) =>                          // 1.1      p ⟶ q
            ⟶.I[P,  ⊥]{ `1.1.1`: P =>                   // 1.1.1    p
                val `1.1.1.1`: ¬[Q] = ⟶.E(`1`)(`1.1.1`) // 1.1.1.1  ¬q
                val `1.1.1.2`: Q = ⟶.E(`1.1`)(`1.1.1`)  // 1.1.1.2  q
                ⟶.E[Q, ⊥](`1.1.1.1`)(`1.1.1.2`):  ⊥     // 1.1.1.3  ⊥
            }: ¬[P]                                     // 1.1.2    ¬p
        }: (P ⟶ Q) ⟶ ¬[P]                               // 1.2      (p ⟶ q) ⟶ ¬q
    }: (P ⟶ ¬[Q]) ⟶ ((P ⟶ Q) ⟶ ¬[P])                    // 2        (p ⟶ ¬q) ⟶ (p ⟶ q) ⟶ ¬q

defined function proof3

# $\vdash (p \rightarrow q) \rightarrow (\neg q \rightarrow \neg p)$

In [145]:
def program[P, Q]: (P => Q) => (Not[Q] => Not[P]) = 
    (ifpq: P => Q) => 
        (nq: (Q => Absurd)) => 
            (p: P) => 
                 nq(ifpq(p)): Absurd

defined function program

The previous examples were all positive ones: if we have a theorem, then we can implement the corresponding function. The opposite also works: if some proposition is not a theorem, then we won't be able to implement that function, no matter how hard we try.

# The intuitionistic stance

But, what about the following theorem? 

# $\vdash \neg (p \wedge q) \rightarrow \neg p \vee \neg q$

In [133]:
def program[P, Q]: Not[(P, Q)] => Either[Not[P], Not[Q]] = 
    { `1`: (((P, Q)) => Nothing) => 
        // Left((p: P) => `1`((p : P, ??? : Q)) : Nothing)
        Right((q: Q) => `1`((??? : P, q : Q)) : Nothing)
    }

defined function program

No matter how hard we try, we get stuck, i.e. we can't complete an implementation of that function. And yet, the proposition is a theorem of logic ... or is it not? Well, it depends on the kind of _logic_ we are talking about: indeed, that formula is a theorem of _classical_ logic, but it's is not a theorem of _intuitionistic_ logic. And the kind of logic which actually corresponds to the kind of functional programming we are doing is _intuitionistic propositional logic_. This logic is characterised by a different mode of reasoning: in classical logic, our reasoning revolves around _truth_, whereas in intuitionistic logic our reasoning revolves around _proofs_. For instance, the formula above can be read as follows:

* If p and q is false, then p is false or q is false (the classical reading)

* If we can prove absurdity given a proof of p and a proof of q, then we can prove absurdity given either a proof of p alone, or a proof of q alone (the intuitionistic reading)

Clasically, it makes sense; intuitionistically, the premise does not support the conclusion at all. The same happens with other formulas, most notably, the double negation theorem and the law of the third middle: 

# $\vdash \neg \neg p \rightarrow p$

In [134]:
def program[P]: Not[Not[P]] => P = 
    { n : ((P => Nothing) => Nothing) => 
        ??? : P 
    }

defined function program

In this case, the premise asserts that we can reach a contradiction (proving the absurd) if we asume that p can't be proved; but that does not mean that we can construct an actual proof of $p$ at all!

# $\vdash p \vee \neg p$

In [135]:
def program[P]: Either[P, Not[P]] = 
    ??? 
    // Left(??? : P)
    // Right((p: P) => ??? : Nothing)

defined function program

"Given any proposition $p$, we can construct a proof of $p$ or show that a proof of $p$ leads to contradiction" ... Come off it!

These two formulas are important, since we can obtain the expressiveness of classical logic if we assume one of them as premise. For instance:

In [143]:
def program[P, Q](thirdMiddleP: Either[P, Not[P]]): Not[(P, Q)] => Either[Not[P], Not[Q]] = 
    { `1`: (((P, Q)) => Nothing) => 
        thirdMiddleP match {
            case Left(p: P) => 
                Right((q: Q) => `1`((p : P, q : Q)) : Nothing)
            case Right(np: Not[P]) => 
                Left((p: P) => np(p) : Nothing)
        }
    }


defined function program

# Conclusion


The correspondence between logic and functional programming is striking, and it doesn't constrain to the type of functional programming and logic that we have exposed. It's something that pervades the whole computation spectrum of computation and logics. Right now, the computation landscape is plagued with programming languages in which this analogy is not apparent, but the time will come when programming practice will be logic. Logic will prevail!

In [6]:
def proof[P, Q]: P => Q => P = 
    (p: P) => (q: Q) => p : P 

defined function proof

$X*(Y+Z) \cong X*Y + X*Z$

$\vdash X \wedge (Y \vee Z) \leftrightarrow X \wedge Y \vee X \wedge Z$

In [85]:
def proof[P, Q, R]: (P ∧ (Q ∨ R)) ⟷ ((P ∧ Q) ∨ (P ∧ R)) = 
    ∧.I(
        ⟶.I{ a: P ∧ (Q ∨ R) => 
            ∨.E((q: Q) => ∨.IL(∧.I(∧.EL(a), q)),
                (r: R) => ∨.IR(∧.I(∧.EL(a), r)))(∧.ER(a))
        },
        ⟶.I{ 
            ∨.E((a: P ∧ Q) => ∧.I(∧.EL(a), ∨.IL(∧.ER(a))),
                (a: P ∧ R) => ∧.I(∧.EL(a), ∨.IR(∧.ER(a))))
        }
    )

defined function proof

In [86]:
def proof[P, Q, R]: (P, Either[Q, R]) <=> Either[(P, Q), (P, R)] = 
    ({ case (p, Left(q)) => Left((p,q))
        case (p, Right(r)) => Right((p,r)) },
    { case Left((p,q)) => (p, Left(q))
        case Right((p,r)) => (p, Right(r)) })

defined function proof

$X^{Y+Z} \cong X^Y * X^Z$

$\vdash (Y \vee Z \rightarrow X) \leftrightarrow (Y \rightarrow X) \wedge (Z \rightarrow X)$

In [87]:
def proof[X, Y, Z]: (Y ∨ Z => X) <=> (Y => X ∧ Z => X) = 
    (??? : (Y ∨ Z => X) => (Y => X ∧ Z => X), 
     ??? : (Y ∨ Z => X) <= (Y => X ∧ Z => X))

defined function proof

In [88]:
def proof[X, Y, Z]: (Either[Y, Z] => X) <=> (Y => X, Z => X) = 
    (f => (y => f(Left(y)), z => f(Right(z))) , 
     { case (f, g) => 
         { case Left(y) => f(y)
           case Right(z) => g(z)
         }
     })

defined function proof

In [29]:

def proof8[A, B]: Or[A, B] => Or[B, A] = 
    Or.elim[A, B, Or[B, A]](Or.applyR[B, A], Or.applyL[B, A])

// Not and implies

def proof3[A, B]: (A => Not[B]) => (A => B) => Not[A] = 
    i => (i2: A => B) => (a: A) => i(a)(i2(a))

def proof2[A, B]: Not[A] => A => B = 
    n => Not.elim[A, B](n)

def proof4[A, B]: Not[A => B] => Not[B] = 
    n => (b: B) => n((_: A) => b)

// Not and and

def proof5[A, B]: Not[A] => Not[And[A, B]] = 
//    (and: And[A, B]) => n(And.elimL(and))
    n => n compose And.elimL

def proof7[A, B]: Not[And[A, B]] => A => Not[B] = 
    n => (a: A) => (b: B) => n(And(a, b))

def proof11[A]: Not[A And Not[A]] = 
    a => And.elimR(a)(And.elimL(a))

// Not and or

def proof9[A, B]: Not[A Or B] => Not[A] And Not[B] = 
    n => And((a: A) => n(Or.applyL(a)), (b: B) => n(Or.applyR(b)))

def proof10[A, B]: (Not[A] And Not[B]) => Not[A Or B] = 
    and => Or.elim[A, B, Absurd](And.elimL(and), And.elimR(and))



defined function proof8
defined function proof3
defined function proof2
defined function proof4
defined function proof41
defined function proof5
defined function proof7
defined function proof11
defined function proof9
defined function proof10

# $\vdash \neg \neg \neg p \leftrightarrow \neg p$

In [138]:
def proof[P]: Not[Not[Not[P]]] <=> Not[P] = 
    ((nnnp: ((P => Absurd) => Absurd) => Absurd) => 
        (p: P) => 
            nnnp( (np : P => Absurd) => np(p) : Absurd): Absurd,
     (np: (P => Absurd)) => 
         (nnp : ((P => Absurd) => Absurd)) => nnp(np) : Absurd)

defined function proof

In [13]:
trait ClassicalLogic{
    trait DN{
        def apply[A](nn: Not[Not[A]]): A
    }
    
    val dn: DN
    
    def proof1[A]: (Not[A] => A) => A =
        f => dn((n: Not[A]) => proof11(And(f(n),n)))
}


defined trait ClassicalLogic

All P are S
All S are R
All P are R

In [15]:
def proof[P, S, R]: ((P => S, S => R)) => P => R = 
    t => (p: P) => t._2(t._1(p))

defined function proof

AG. $\{ r \vee p \rightarrow \neg p, p \rightarrow q, \neg q \vee p\} \vdash \neg p$

In [113]:
def proof[P, Q, R](
    p1: R Or P => Not[P], 
    p2: P => Q, 
    p3: Not[Q] Or P): Not[P] = 
    (p: P) => p1(Right(p))(p) : Absurd

defined function proof

$ \vdash (p \rightarrow (q \rightarrow r)) \rightarrow (p \rightarrow q) \rightarrow p \rightarrow r$

In [38]:
def term[P, Q, R]: (P => (Q => R)) => (P => Q) => (P => R) = 
    (p1: P => Q => R) => 
        (p2: P => Q) => 
            (p: P) => p1(p : P)(p2(p) : Q) : R

defined function term

In [94]:
def proof[P, Q, R]: (P ⟶ (Q ⟶ R)) ⟶ ((P ⟶ Q) ⟶ (P ⟶ R)) = 
    ⟶.I{ p1: P ⟶ (Q ⟶ R) => 
        ⟶.I{ p2: P ⟶ Q => 
            ⟶.I{ p: P =>
                ⟶.E(⟶.E(p1)(p))(⟶.E(p2)(p))
            }
        }
    }

defined function proof

$ \vdash p \rightarrow q \rightarrow p$

In [39]:
def proof[P, Q, R]: P => Q => P = 
    (p: P) => 
        (q: Q) => p : P

defined function proof

In [96]:
def proof[P, Q, R]: P ⟶ (Q ⟶ P) = 
    ⟶.I{ p: P => 
        ⟶.I{ q: Q => 
            p : P
        }
    }

defined function proof

In [98]:
def proof[A, B, C]: (A => (B => C)) => (A Either B => C) = 
    (p: A => B => C) => 
        (e: Either[A, B]) => 
            e.fold(
                (a: A) => ??? : C, 
                (b: B) => ??? : C)

defined function proof

Ejercicio AG.

In [107]:
def proof[P, Q, R, S, T](
    p1: P => Not[R] Or S,
    p2: Not[R] => S And Not[T],
    p3: P => T,
    p4: P => Not[S]): Not[P] = 
    (p: P) => 
        p1(p) match {
            case nr: Not[R] => p4(p)(p2(nr)._1) : Absurd
            case s: S => p4(p)(s) : Absurd
        }

defined function proof

In [112]:
def proof[P, Q, R, S, T](
    p1: P ⟶ (¬[R] ∨ S),
    p2: ¬[R] ⟶ (S ∧ ¬[T]),
    p3: P ⟶ T,
    p4: P ⟶ ¬[S]): ¬[P] = 
    ⟶.I((p: P) => 
        ∨.E(
            (nr: Not[R]) => 
                ⟶.E(⟶.E(p4)(p))(∧.EL(⟶.E(p2)(nr))) : Absurd,
            (s: S) => 
                ⟶.E(⟶.E(p4)(p))(s) : Absurd
        )(⟶.E(p1)(p)))

defined function proof